In [2]:
from pprint import pprint
import time
import ee
import folium
import tensorflow as tf
from tensorflow import keras
import json

In [3]:
print(tf.__version__)

2.1.0


In [3]:
ee.Authenticate()

Enter verification code:  4/vgEExIZaXcSrMMdojwxx_1FRy0akolieYQh5ze8jrJQf4Kuu6-DYEYg



Successfully saved authorization token.


In [4]:
ee.Initialize()

In [5]:
# Use these bands for prediction.
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']
# Use Landsat 8 surface reflectance data.
l8sr = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')

# Cloud masking function.
def maskL8sr(image):
  cloudShadowBitMask = ee.Number(2).pow(3).int()
  cloudsBitMask = ee.Number(2).pow(5).int()
  qa = image.select('pixel_qa')
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(
    qa.bitwiseAnd(cloudsBitMask).eq(0))
  return image.updateMask(mask).select(bands).divide(10000)

# The image input data is a 2018 cloud-masked median composite.
image = l8sr.filterDate('2018-01-01', '2018-12-31').map(maskL8sr).median()

# Use folium to visualize the imagery.
mapIdDict = image.getMapId({'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3})
map = folium.Map(location=[45.4, -75.7])
folium.TileLayer(
    tiles=mapIdDict['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='median composite',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

In [6]:
# Sample the image at the points and add a random column.
sampleRegion = ee.Geometry.Rectangle([-139, 48.5, -114, 60])
sample = image.sample(region=sampleRegion, scale=10000).randomColumn()

# Partition the sample approximately 70-30.
training = sample.filter(ee.Filter.lt('random', 0.7))
testing = sample.filter(ee.Filter.gte('random', 0.7))

# This is list of all the properties we want to export.
featureNames = list(bands)

In [ ]:
# Create the tasks.
testingTaskCSV = ee.batch.Export.table.toDrive(
  collection=testing,
  description='Testing Export',
  folder='TFData',
  fileFormat='csv',
  selectors=featureNames)

testingTaskTFRecord = ee.batch.Export.table.toDrive(
  collection=testing,
  description='Testing Export',
  folder='TFData',
  fileFormat='TFRecord',
  selectors=featureNames)

In [ ]:
testingTaskCSV.start()
testingTaskTFRecord.start()

In [ ]:
while testingTaskTFRecord.active():
    print(ee.data.getTaskStatus(testingTaskCSV.id))
    print(ee.data.getTaskStatus(testingTaskTFRecord.id))
    time.sleep(5)
print('Done with testing export.')

In [ ]:
print(ee.batch.Task.list())

In [ ]:
trainingTaskCSV = ee.batch.Export.table.toDrive(
  collection=training,
  description='Training Export',
  folder='TFData',
  fileFormat='csv',
  selectors=featureNames)

trainingTaskTFRecord = ee.batch.Export.table.toDrive(
  collection=training,
  description='Training Export',
  folder='TFData',
  fileFormat='TFRecord',
  selectors=featureNames)

In [ ]:
trainingTaskCSV.start()
trainingTaskTFRecord.start()

In [ ]:
while trainingTaskTFRecord.active():
  print(ee.data.getTaskStatus(trainingTaskCSV.id))
  print(ee.data.getTaskStatus(trainingTaskTFRecord.id))
  time.sleep(5)
print('Done with training export.')

In [ ]:
print(ee.batch.Task.list())

In [11]:
# Export imagery in this region.
exportRegionBC = ee.Geometry.Rectangle([-122, 58, -121, 59])
exportRegionSahara = ee.Geometry.Rectangle([13, 24, 14, 25])

exportBC = image.sample(region=exportRegionBC, scale=100)
exportSahara = image.sample(region=exportRegionSahara, scale=100)

# Setup the task.
BCimageTaskCSV = ee.batch.Export.table.toDrive(
  collection=exportBC,
  description='BC Image Export',
  folder='TFData',
  fileFormat='csv',
  selectors=featureNames
)

SaharaImageTaskCSV = ee.batch.Export.table.toDrive(
  collection=exportSahara,
  description='Sahara Image Export',
  folder='TFData',
  fileFormat='csv',
  selectors=featureNames,
)

In [12]:
# Start the task.
BCimageTaskCSV.start()
SaharaImageTaskCSV.start()

In [ ]:
while SaharaImageTaskTFRecord.active():
  print(ee.data.getTaskStatus(BCimageTaskTFRecord.id))
  print(ee.data.getTaskStatus(SaharaImageTaskTFRecord.id))
  time.sleep(5)
print('Done with image export.')

In [7]:
print(ee.batch.Task.list())

[<Task EXPORT_FEATURES: Sahara Image Export (COMPLETED)>, <Task EXPORT_FEATURES: BC Image Export (COMPLETED)>, <Task EXPORT_IMAGE: Sahara Image Export (FAILED)>, <Task EXPORT_IMAGE: BC Image Export (COMPLETED)>, <Task EXPORT_IMAGE: Image Export (FAILED)>, <Task EXPORT_IMAGE: Image Export (CANCELLED)>, <Task EXPORT_IMAGE: Image Export (FAILED)>, <Task EXPORT_IMAGE: Image Export (CANCELLED)>, <Task EXPORT_IMAGE: Image Export (FAILED)>, <Task EXPORT_IMAGE: Image Export (COMPLETED)>, <Task EXPORT_FEATURES: Training Export (COMPLETED)>, <Task EXPORT_FEATURES: Training Export (COMPLETED)>, <Task EXPORT_FEATURES: Testing Export (COMPLETED)>, <Task EXPORT_FEATURES: Testing Export (COMPLETED)>, <Task EXPORT_FEATURES: Training Export (COMPLETED)>, <Task EXPORT_FEATURES: Testing Export (COMPLETED)>, <Task EXPORT_FEATURES: Testing Export (FAILED)>, <Task EXPORT_IMAGE: Image Export (COMPLETED)>, <Task EXPORT_FEATURES: Testing Export (FAILED)>, <Task EXPORT_FEATURES: Testing Export (FAILED)>, <Task 